In [88]:
from andi_datasets.models_phenom import models_phenom
from helpersMSD import *
from helpersGeneration import *
from helpersPlot import *
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
# number of time steps per trajectory (frames)
T = 200
# number of trajectories
N = 1
# Length of box (pixels), set to 0 to make them start at 0,0
L = 0 
# diffusion coefficient (pixels^2 / frame)
D = 2

In [51]:
trajs, labels = models_phenom().single_state(N, 
                                L = L,
                                T = T,
                                Ds = [D, 0.0], # Mean and variance
                                alphas = 1)
print(trajs.shape,labels.shape)
# Need to reshape generated trajectories because they are in format (T,N,dim), but we want them in (N,T,dim)
trajs = trajs.transpose(1,0,2)
labels = labels.transpose(1,0,2)
print(trajs.shape,labels.shape)
# Labels are now in format (N,T,3): for each particle T times the tuple (alpha, D, state)
print(f"Particle 0 [alpha, Gen_D, state]: {labels[0,0,:]}")


(200, 1, 2) (200, 1, 3)
(1, 200, 2) (1, 200, 3)
Particle 0 [alpha, Gen_D, state]: [1. 2. 2.]


In [105]:
# Hyperparameters for simulation
nframes = 20    # Number of steps in the simulation
nposframe = 10    # Number of position per frame
num_steps = nframes*nposframe
# Hyperparameters for image generation
npixel = 32 # number of image pixels
pixelsize = 100 # in nm 
fwhm_psf = 200 # full width half maximum (emulates microscope)
factor_hr = 5 # image high resulution factor
flux = 50 # number of photons per s
poisson_noise = 100 
gaussian_noise = 15
background = 1000 # base background value

frames_hr, frames_lr, frames_noisy = generateImages(trajs[0],nframes,npixel,factor_hr,nposframe,fwhm_psf,pixelsize,flux,background,gaussian_noise)
print(frames_hr.shape)
frames_hr, frames_lr, frames_noisy = np.expand_dims(frames_hr,axis=-1),np.expand_dims(frames_lr,axis=-1),np.expand_dims(frames_noisy,axis=-1)

print(frames_hr.shape)

(20, 160, 160)
(20, 160, 160, 1)


In [104]:
play_video(frames_hr,save_path="frames_hr.mp4")

vmin: 3.7804086747129308e-208 vmax: 444.8079555958928


Animation saved to frames_hr.mp4


In [103]:
play_video(frames_lr,save_path="frames_lr.mp4")

vmin: 8.600376479772272e-194 vmax: 409.3842129612242


Animation saved to frames_lr.mp4


In [107]:
play_video(frames_noisy,save_path="frames_noisy.gif")

vmin: 940.2012624420144 vmax: 1402.2528441834356


Animation saved to frames_noisy.gif


In [87]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

def animate_particle_trajectory(trajectory, nframes, points_per_frame=None, fps=5, save_path=None):
    """
    Creates an animation of a particle's trajectory, showing one subtrajectory per frame.
    No axis labels or ticks for a clean visualization.
    
    Parameters:
    - trajectory: np.ndarray of shape (N, 2), where N is the total number of points.
                 Each row represents the (x, y) coordinates of the particle.
    - nframes: int, number of frames to divide the trajectory into.
    - points_per_frame: int, optional, number of points to show in each frame.
                       If None, it's calculated as len(trajectory) // nframes.
    - fps: int, frames per second for the animation.
    
    Returns:
    - HTML animation that can be displayed in a Jupyter notebook.
    """
    # Set up the figure and axis
    fig, ax = plt.subplots(figsize=(5, 5))
    
    # Calculate points per frame if not provided
    if points_per_frame is None:
        points_per_frame = len(trajectory) // nframes
    
    # Set consistent axis limits
    x_min, x_max = np.min(trajectory[:, 0]), np.max(trajectory[:, 0])
    y_min, y_max = np.min(-trajectory[:, 1]), np.max(-trajectory[:, 1])
    
    # Find the maximum absolute value to create a symmetric plot
    max_extent = max(
        abs(x_min), 
        abs(x_max), 
        abs(y_min), 
        abs(y_max)
    )
    
    # Add some padding to the limits
    max_extent = max_extent * 1.1  # Add 10% padding
    
    # Set symmetric limits centered at origin
    ax.set_xlim([-max_extent, max_extent])
    ax.set_ylim([-max_extent, max_extent])
    
    # Turn off all axis labeling
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    

    # Set up plot elements
    ax.set_aspect('equal')
    
    # Initialize empty line and text objects
    line, = ax.plot([], [], lw=2)
    frame_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, fontsize=12)
    
    # Initialize empty arrays for accumulated trajectory
    x_data, y_data = [], []
    
    def init():
        """Initialize animation"""
        line.set_data([], [])
        frame_text.set_text('')
        return line, frame_text
    
    def update(frame):
        """Update animation for each frame"""
        # Calculate start and end indices for this frame
        start = frame * points_per_frame
        end = min(start + points_per_frame, len(trajectory))
        
        # Get the current trajectory segment
        current_segment = trajectory[start:end]
        
        # Append to accumulated data
        if frame == 0:
            x_data.clear()
            y_data.clear()
        
        x_data.extend(current_segment[:, 0])
        y_data.extend(-current_segment[:, 1])  # Negative y to match your original plot
        
        # Update the line data
        line.set_data(x_data, y_data)
        
        # Update the frame text
        frame_text.set_text(f'Frame {frame + 1}/{nframes}')
        
        return line, frame_text
    
    # Create the animation
    ani = animation.FuncAnimation(fig, update, frames=nframes,
                                 init_func=init, blit=True, interval=1000/fps)
    # Save the animation if a save path is provided
    if save_path:
        if save_path.endswith('.mp4'):
            # Use FFMpegWriter for MP4 files (requires FFmpeg installed)
            writer = animation.FFMpegWriter(fps=fps, metadata=dict(artist='Me'), bitrate=1800)
            ani.save(save_path, writer=writer)
            print(f"Animation saved to {save_path}")
        elif save_path.endswith('.gif'):
            # Use PillowWriter for GIF files
            writer = animation.PillowWriter(fps=fps)
            ani.save(save_path, writer=writer)
            print(f"Animation saved to {save_path}")
        else:
            print("Unsupported file format. Use .mp4 or .gif extension.")
    plt.close()  # Prevent displaying the static plot
    
    return HTML(ani.to_jshtml())

# Example usage:
animate_particle_trajectory(trajs[0], nframes=20, save_path="TrajectoryBlue.gif")

Animation saved to TrajectoryBlue.gif


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from matplotlib.cm import tab20

def animate_particle_trajectory(trajectory, nframes, points_per_frame=None, fps=5, save_path=None):
    """
    Creates an animation of a particle's trajectory, showing one subtrajectory per frame
    with each frame in a different color using the tab20 colormap.
    
    Parameters:
    - trajectory: np.ndarray of shape (N, 2), where N is the total number of points.
                 Each row represents the (x, y) coordinates of the particle.
    - nframes: int, number of frames to divide the trajectory into.
    - points_per_frame: int, optional, number of points to show in each frame.
                       If None, it's calculated as len(trajectory) // nframes.
    - fps: int, frames per second for the animation.
    
    Returns:
    - HTML animation that can be displayed in a Jupyter notebook.
    """
    # Set up the figure and axis
    fig, ax = plt.subplots(figsize=(6, 6))
    
    # Calculate points per frame if not provided
    if points_per_frame is None:
        points_per_frame = len(trajectory) // nframes
    
    # Set consistent axis limits
    x_min, x_max = np.min(trajectory[:, 0]), np.max(trajectory[:, 0])
    y_min, y_max = np.min(-trajectory[:, 1]), np.max(-trajectory[:, 1])
    
    # Find the maximum absolute value to create a symmetric plot
    max_extent = max(
        abs(x_min), 
        abs(x_max), 
        abs(y_min), 
        abs(y_max)
    )
    
    # Add some padding to the limits
    max_extent = max_extent * 1.1  # Add 10% padding
    
    # Set symmetric limits centered at origin
    ax.set_xlim([-max_extent, max_extent])
    ax.set_ylim([-max_extent, max_extent])
    
    # Set up plot elements
    ax.set_xlabel('X Position (nm)', fontsize=14)
    ax.set_ylabel('Y Position (nm)', fontsize=14)
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.set_aspect('equal')
    
    # Initialize empty line objects for each frame segment
    lines = [ax.plot([], [], lw=2, label=f'Frame {i+1}')[0] for i in range(nframes)]
    frame_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, fontsize=12)
    
    # Get colors from tab20 colormap
    colors = tab20.colors[:nframes] if nframes <= 20 else [tab20(i % 20) for i in range(nframes)]
    
    # Set colors for each line
    for i, line in enumerate(lines):
        line.set_color(colors[i])
    
    # Create empty segments data
    segments_data = [[] for _ in range(nframes)]
    
    def init():
        """Initialize animation"""
        for line in lines:
            line.set_data([], [])
        frame_text.set_text('')
        return lines + [frame_text]
    
    def update(frame):
        """Update animation for each frame"""
        current_frame = frame
        
        # Show lines for all frames up to the current one
        for i in range(current_frame + 1):
            if len(segments_data[i]) == 0:  # If this segment hasn't been calculated yet
                # Calculate start and end indices for this segment
                start = max(0,i * points_per_frame -1)
                end = min(start + points_per_frame+1, len(trajectory))
                
                # Store the segment data
                segments_data[i] = (
                    trajectory[start:end, 0],
                    -trajectory[start:end, 1]  # Negative y to match original plot
                )
            
            # Set the data for this line
            lines[i].set_data(*segments_data[i])
        
        # Hide lines for future frames
        for i in range(current_frame + 1, nframes):
            lines[i].set_data([], [])
        
        # Update the frame text
        frame_text.set_text(f'Frame {current_frame + 1}/{nframes}')
        ax.set_title(f'Simulated Trajectory - Frame {current_frame + 1}/{nframes}')

    
        
        return lines + [frame_text]
    
    # Create the animation
    ani = animation.FuncAnimation(fig, update, frames=nframes,
                                 init_func=init, blit=True, interval=1000/fps)
    # Save the animation if a save path is provided
    if save_path:
        if save_path.endswith('.mp4'):
            # Use FFMpegWriter for MP4 files (requires FFmpeg installed)
            writer = animation.FFMpegWriter(fps=fps, metadata=dict(artist='Me'), bitrate=1800)
            ani.save(save_path, writer=writer)
            print(f"Animation saved to {save_path}")
        elif save_path.endswith('.gif'):
            # Use PillowWriter for GIF files
            writer = animation.PillowWriter(fps=fps)
            ani.save(save_path, writer=writer)
            print(f"Animation saved to {save_path}")
        else:
            print("Unsupported file format. Use .mp4 or .gif extension.")
    plt.close()  # Prevent displaying the static plot
    
    return HTML(ani.to_jshtml())

# Example usage:
# trajectory = np.random.normal(0, 1, size=(400, 2)).cumsum(axis=0)  # Example random walk
animate_particle_trajectory(trajs[0], nframes=20, save_path='TrajectoryColors.gif')


Animation saved to TrajectoryColors.gif
